In [27]:
import os
import findspark
findspark.init(os.environ['SPARK_HOME'])

from pyspark.sql import SparkSession

# Inicializando a sessão do Spark
spark = SparkSession.builder.appName("Exemplo").getOrCreate()

## Text Cleaning and NLP
---

- **Tabela de frequência:**
* **Relativa**
* Absoluta

In [28]:
import nltk
from nltk.corpus import stopwords


from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import explode, col, lit, when

In [29]:
# Baixar a lista de stopwords em português do NLTK
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aview\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
# Obter a lista de stopwords em português
stopwords_portugues = set(stopwords.words('portuguese'))

In [31]:
# Total de stopwords em portugues: 
print(f"Total de stopwords em portugues: {len(stopwords_portugues)}")

Total de stopwords em portugues: 207


In [32]:
inputPath = "..\data\clientes.csv"

# Carregue os dados
data_cliente = spark \
        .read\
        .format("csv")\
        .option("inferSchema", "true")\
        .option("header", "true")\
        .option("encoding", "UTF-8")\
        .csv(inputPath) \
        
data = data_cliente.filter("NM_CIDADE IS NOT NULL")
data.show(truncate=False)

+------------------------------+--------+----------------------+-----+-------------------+-------------------+------------------+--------------+---------------+----------+
|EN_RUA                        |NR_CEP  |NM_CIDADE             |SG_UF|NM_BAIRRO          |DS_LATITUDE        |DS_LONGITUDE      |CPF           |NOME           |CD_CLIENTE|
+------------------------------+--------+----------------------+-----+-------------------+-------------------+------------------+--------------+---------------+----------+
|RUA EXISTENTE                 |09340060|MAUA                  |SP   |VILA ANA           |NULL               |NULL              |595.958.963-00|Beatriz Silva  |0         |
|AVENIDA DOS TRABALHADORES     |13338050|INDAIATUBA            |SP   |VILA CASTELO BRANCO|-23.0887183        |-47.198916        |695.935.479-00|Maria Ferreira |1         |
|R OTAVIO DE MORAIS            |05351080|S PAULO               |SP   |C S FRANCISCO      |NULL               |NULL              |223.379.779

In [33]:
total_rows = data.count()
total_rows

3298430

In [34]:
# Corrigindo nomes de cidades com grafias incorretas
data = data.withColumn("NM_CIDADE", 
                            when(data["NM_CIDADE"].like("%S PAULO%"), "SAO PAULO")
                            .when(data["NM_CIDADE"].like("%STO AMARO%"), "SAO PAULO")
                            .when(data["NM_CIDADE"].like("%S BERNARDO CAMPO%"), "SAO BERNARDO DO CAMPO")
                            .when(data["NM_CIDADE"].like("%S BERNARDO DO CAMPO%"), "SAO BERNARDO DO CAMPO")
                            .when(data["NM_CIDADE"].like("%S B DO CAMPO%"), "SAO BERNARDO DO CAMPO")
                            .when(data["NM_CIDADE"].like("%S B CAMPO%"), "SAO BERNARDO DO CAMPO")
                            .when(data["NM_CIDADE"].like("%S JOSE R PRETO%"), "SAO JOSE DO RIO PRETO")
                            .when(data["NM_CIDADE"].like("%S JOSE DO RIO PRETO%"), "SAO JOSE DO RIO PRETO")
                            .when(data["NM_CIDADE"].like("%S J DO RIO PRETO%"), "SAO JOSE DO RIO PRETO")
                            .when(data["NM_CIDADE"].like("%S J DO RIO PARDO%"), "SAO JOSE DO RIO PARDO")
                            .when(data["NM_CIDADE"].like("%S J  DO RIO PRETO%"), "SAO JOSE DO RIO PRETO")
                            .when(data["NM_CIDADE"].like("%S JOSE CAMPOS%"), "SAO JOSE DOS CAMPOS")
                            .when(data["NM_CIDADE"].like("%S J CAMPOS%"), "SAO JOSE DOS CAMPOS")
                            .when(data["NM_CIDADE"].like("%S VICENTE%"), "SAO VICENTE")
                            .when(data["NM_CIDADE"].like("%S ROQUE%"), "SAO ROQUE")
                            .when(data["NM_CIDADE"].like("%S CAETANO SUL%"), "SAO CAETANO DO SUL")
                            .when(data["NM_CIDADE"].like("%S C DO SUL%"), "SAO CAETANO DO SUL")
                            .when(data["NM_CIDADE"].like("%S JOAO B VISTA%"), "SAO JOAO DA BOA VISTA")
                            .when(data["NM_CIDADE"].like("%S JOAO DA BOA VISTA%"), "SAO JOAO DA BOA VISTA")
                            .when(data["NM_CIDADE"].like("%S CARLOS%"), "SAO CARLOS")
                            .when(data["NM_CIDADE"].like("%S  CARLOS%"), "SAO CARLOS")
                            .when(data["NM_CIDADE"].like("%S SEBASTIAO%"), "SAO SEBASTIAO")
                            .when(data["NM_CIDADE"].like("%E S DO PINHAL%"), "ESPIRITO SANTO DO PINHAL")
                            .when(data["NM_CIDADE"].like("%E S PINHAL%"), "ESPIRITO SANTO DO PINHAL")
                            .when(data["NM_CIDADE"].like("%ESPIRITO STO DO PINHAL%"), "ESPIRITO SANTO DO PINHAL")
                            .when(data["NM_CIDADE"].like("%ESP STO DO PINHAL%"), "ESPIRITO SANTO DO PINHAL")
                            .when(data["NM_CIDADE"].like("%ESPIRITO STO PINHAL%"), "ESPIRITO SANTO DO PINHAL")
                            .when(data["NM_CIDADE"].like("%E STO DO PINHAL%"), "ESPIRITO SANTO DO PINHAL")
                            .when(data["NM_CIDADE"].like("%ESPIRITO STO  PINHAL%"), "ESPIRITO SANTO DO PINHAL")
                            .when(data["NM_CIDADE"].like("%ESP STO PINHAL%"), "ESPIRITO SANTO DO PINHAL")
                            .when(data["NM_CIDADE"].like("%ESPIROTO STO DO PINHAL%"), "ESPIRITO SANTO DO PINHAL")
                            .when(data["NM_CIDADE"].like("%ESP STO  DO PINHAL%"), "ESPIRITO SANTO DO PINHAL")
                            .when(data["NM_CIDADE"].like("%ESPIRITO STO DOPINHAL%"), "ESPIRITO SANTO DO PINHAL")
                            .when(data["NM_CIDADE"].like("%ESP  STO  DO PINHAL%"), "ESPIRITO SANTO DO PINHAL")
                            .when(data["NM_CIDADE"].like("%ESPIRITO STO DE PINHAL%"), "ESPIRITO SANTO DO PINHAL")
                            .when(data["NM_CIDADE"].like("%ESPIRITO STO DO TURVO%"), "ESPIRITO SANTO DO TURVO")
                            .when(data["NM_CIDADE"].like("%STO ANDRE%"), "SANTO ANDRE")
                            .when(data["NM_CIDADE"].like("%STO  ANDRE%"), "SANTO ANDRE")
                            .when(data["NM_CIDADE"].like("%STO   ANDRE%"), "SANTO ANDRE")
                            .when(data["NM_CIDADE"].like("%STO ANTONIO DO PINHAL%"), "SANTO ANTONIO DO PINHAL")
                            .when(data["NM_CIDADE"].like("%STO ANTONIO ARACANGUA%"), "SANTO ANTONIO DO ARACANGUA")
                            .when(data["NM_CIDADE"].like("%STO ANT DO ARACANGUA%"), "SANTO ANTONIO DO ARACANGUA")
                            .when(data["NM_CIDADE"].like("%STO ANT ARACANGUA%"), "SANTO ANTONIO DO ARACANGUA")
                            .when(data["NM_CIDADE"].like("%STO ANASTACIO%"), "SANTO ANASTACIO")
                            .when(data["NM_CIDADE"].like("%STO ANTONIO DO JARDIM%"), "SANTO ANTONIO DO JARDIM")
                            .when(data["NM_CIDADE"].like("%STO  ANTO  JARDIM%"), "SANTO ANTONIO DO JARDIM")
                            .when(data["NM_CIDADE"].like("%STO ANT DO JARDIM%"), "SANTO ANTONIO DO JARDIM")
                            .when(data["NM_CIDADE"].like("%STO ANTONIO DE POSSE%"), "SANTO ANTONIO DE POSSE")
                            .otherwise(data["NM_CIDADE"]))

In [35]:
# Tokenize os textos
tokenizer = Tokenizer(inputCol="NM_CIDADE", outputCol="palavras")
tokenized = tokenizer.transform(data)
tokenized.show(truncate=False)

+------------------------------+--------+----------------------+-----+-------------------+-------------------+------------------+--------------+---------------+----------+---------------------------+
|EN_RUA                        |NR_CEP  |NM_CIDADE             |SG_UF|NM_BAIRRO          |DS_LATITUDE        |DS_LONGITUDE      |CPF           |NOME           |CD_CLIENTE|palavras                   |
+------------------------------+--------+----------------------+-----+-------------------+-------------------+------------------+--------------+---------------+----------+---------------------------+
|RUA EXISTENTE                 |09340060|MAUA                  |SP   |VILA ANA           |NULL               |NULL              |595.958.963-00|Beatriz Silva  |0         |[maua]                     |
|AVENIDA DOS TRABALHADORES     |13338050|INDAIATUBA            |SP   |VILA CASTELO BRANCO|-23.0887183        |-47.198916        |695.935.479-00|Maria Ferreira |1         |[indaiatuba]               |


In [36]:
# Configurando o remover de stopwords
stopwords_remover = StopWordsRemover(inputCol="palavras", outputCol="TextoSemStopwords", stopWords=list(stopwords_portugues))

# Aplicando a remoção de stopwords (note que você deve ter a coluna "Tokens" do exemplo anterior)
df_sem_stopwords = stopwords_remover.transform(tokenized)

# Exibindo o DataFrame resultante
print("\nDataFrame Após Remoção de Stopwords:")
df_sem_stopwords.show(truncate=False)


DataFrame Após Remoção de Stopwords:
+------------------------------+--------+----------------------+-----+-------------------+-------------------+------------------+--------------+---------------+----------+---------------------------+-----------------------+
|EN_RUA                        |NR_CEP  |NM_CIDADE             |SG_UF|NM_BAIRRO          |DS_LATITUDE        |DS_LONGITUDE      |CPF           |NOME           |CD_CLIENTE|palavras                   |TextoSemStopwords      |
+------------------------------+--------+----------------------+-----+-------------------+-------------------+------------------+--------------+---------------+----------+---------------------------+-----------------------+
|RUA EXISTENTE                 |09340060|MAUA                  |SP   |VILA ANA           |NULL               |NULL              |595.958.963-00|Beatriz Silva  |0         |[maua]                     |[maua]                 |
|AVENIDA DOS TRABALHADORES     |13338050|INDAIATUBA            |SP

In [37]:
# Exploda as palavras em linhas
exploded = df_sem_stopwords.select(explode("TextoSemStopwords").alias("palavra"), "TextoSemStopwords", "NM_CIDADE")
exploded.groupBy(["palavra", "TextoSemStopwords", "NM_CIDADE"]).count().orderBy(col("count").desc()).show(truncate=False)

+---------------+----------------------+---------------------+------+
|palavra        |TextoSemStopwords     |NM_CIDADE            |count |
+---------------+----------------------+---------------------+------+
|paulo          |[sao, paulo]          |SAO PAULO            |852150|
|sao            |[sao, paulo]          |SAO PAULO            |852150|
|guarulhos      |[guarulhos]           |GUARULHOS            |91668 |
|campinas       |[campinas]            |CAMPINAS             |74731 |
|sao            |[sao, bernardo, campo]|SAO BERNARDO DO CAMPO|59550 |
|campo          |[sao, bernardo, campo]|SAO BERNARDO DO CAMPO|59550 |
|bernardo       |[sao, bernardo, campo]|SAO BERNARDO DO CAMPO|59550 |
|campos         |[sao, jose, campos]   |SAO JOSE DOS CAMPOS  |53257 |
|sao            |[sao, jose, campos]   |SAO JOSE DOS CAMPOS  |53257 |
|jose           |[sao, jose, campos]   |SAO JOSE DOS CAMPOS  |53257 |
|osasco         |[osasco]              |OSASCO               |52714 |
|santo          |[sa

In [38]:
# Conte a frequência das palavras
word_freq = exploded.groupBy("palavra").count().withColumnRenamed("count", "qtd_abs")
word_freq = word_freq.withColumn("qtd_relat", (col("qtd_abs") / lit(total_rows)) * 100).orderBy(col("qtd_relat").desc())

# Mostre a tabela de frequência de palavras
word_freq.show()

+---------+-------+------------------+
|  palavra|qtd_abs|         qtd_relat|
+---------+-------+------------------+
|      sao|1090081|33.048480640789705|
|    paulo| 854853|25.916966556816423|
|guarulhos|  93310| 2.828921638476487|
|     jose|  88767| 2.691189444675194|
| campinas|  75825| 2.298820954211549|
|    preto|  70883| 2.148992096239726|
|    campo|  65673| 1.991038160579427|
| bernardo|  59895| 1.815863911012209|
|   campos|  59606|1.8071021667884417|
|    santo|  56998| 1.728034246596108|
|      rio|  55154|1.6721288613067427|
| ribeirao|  55053|1.6690667984465335|
| paulista|  54155|1.6418417246993267|
|   osasco|  53247|1.6143134764115048|
|     mogi|  48504| 1.470517791797916|
| sorocaba|  48330| 1.465242554791219|
|    andre|  48117| 1.458784937076124|
|    serra|  47390|1.4367441479734298|
|   grande|  37711|1.1433015101123867|
|    santa|  34072|1.0329762947826693|
+---------+-------+------------------+
only showing top 20 rows



In [39]:
word_freq.count()

5046

In [40]:
word_freq.toPandas().to_csv('word_freq.csv')